### 🎯 OpenCV 透视变换与相关函数总结

| 函数 | 功能 | 参数 | 参数说明 | 返回值 | 典型用途 |
|------|------|------|-----------|----------|-----------|
| **`cv2.getPerspectiveTransform(src, dst)`** | 计算透视变换矩阵（3×3） | `src`<br>`dst` | - `src`：原图中四个顶点的坐标（np.float32 类型）<br>- `dst`：目标图像中对应的四个顶点坐标（np.float32 类型） | 3×3 透视变换矩阵 `M` | 根据四对点建立变换关系，用于文档校正、投影变换 |
| **`cv2.warpPerspective(src, M, dsize[, flags[, borderMode[, borderValue]]])`** | 执行透视变换（Perspective Warp） | `src`<br>`M`<br>`dsize`<br>`flags`<br>`borderMode`<br>`borderValue` | - `src`：输入图像<br>- `M`：3×3 透视变换矩阵<br>- `dsize`：输出图像大小 `(width, height)`<br>- `flags`：插值方式（默认 `cv2.INTER_LINEAR`）<br>- `borderMode`：边界填充模式（如 `cv2.BORDER_CONSTANT`）<br>- `borderValue`：边界颜色（默认黑色） | 透视变换后的图像 | 实现视角校正、投影映射、透视效果调整 |
| **`cv2.line(img, pt1, pt2, color, thickness[, lineType])`** | 在图像上绘制直线 | `img`<br>`pt1, pt2`<br>`color`<br>`thickness`<br>`lineType` | - `img`：目标图像<br>- `pt1, pt2`：起点与终点坐标 `(x, y)`<br>- `color`：线条颜色（BGR格式）<br>- `thickness`：线条粗细（像素）<br>- `lineType`：线型（`cv2.LINE_AA` 表示抗锯齿） | 绘制后的图像 | 绘制透视区域、辅助线、边界框 |
| **`cv2.destroyAllWindows()`** | 关闭所有窗口 | 无 | - | 无 | 清理显示窗口资源 |
| **`np.float32(array)`** | 转换数据类型 | `array` | 任意数组 | 转换为 32 位浮点数组 | OpenCV 函数要求输入坐标为 float32 |


In [ ]:
# --------------------------------------------------------
# 🎯 功能说明：
# 本程序演示 OpenCV 的透视变换（Perspective Transformation）
# 将图像中任意四边形区域拉伸/变换为矩形，实现“图像矫正”。
# --------------------------------------------------------

# 1️⃣ 导入必要的库
import cv2
import numpy as np

# --------------------------------------------------------
# 2️⃣ 读取原始图像
# cv2.imread(filename, flags)
#   filename：图像路径
#   flags：
#       cv2.IMREAD_COLOR     以彩色方式读取（默认）
#       cv2.IMREAD_GRAYSCALE 以灰度方式读取
#       cv2.IMREAD_UNCHANGED  读取所有通道（包含透明度）
# --------------------------------------------------------
image_np = cv2.imread("../image/jiaozheng.png")

# 获取图像的高、宽、通道数信息
image_shape = image_np.shape  # (height, width, channels)

# --------------------------------------------------------
# 3️⃣ 定义原图与目标图的顶点坐标
# 透视变换需要 4 对点的映射关系
# --------------------------------------------------------

# 原图像中选取的四个顶点（对应要矫正的四边形区域）
# np.float32(...)：确保坐标为32位浮点型（OpenCV要求）
points = np.float32(
    [
        [118, 60],  # 左上角点
        [350, 90],  # 右上角点
        [75, 195],  # 左下角点
        [338, 230],  # 右下角点
    ]
)

# 目标图像中四个点的位置（拉伸到整个图像大小）
points2 = np.float32(
    [
        [0, 0],  # 左上角
        [image_shape[1], 0],  # 右上角
        [0, image_shape[0]],  # 左下角
        [image_shape[1], image_shape[0]],  # 右下角
    ]
)

# --------------------------------------------------------
# 4️⃣ 绘制辅助线，帮助可视化透视前的四边形区域
# cv2.line(img, pt1, pt2, color, thickness, lineType)
#   img：目标图像
#   pt1, pt2：起点和终点坐标 (x, y)
#   color：(B, G, R)
#   thickness：线条粗细
#   lineType：线型（cv2.LINE_AA 表示抗锯齿线）
# --------------------------------------------------------
cv2.line(image_np, [118, 60], [350, 90], (0, 0, 255), 1, lineType=cv2.LINE_AA)
cv2.line(image_np, [75, 195], [338, 230], (0, 0, 255), 1, lineType=cv2.LINE_AA)

# --------------------------------------------------------
# 5️⃣ 计算透视变换矩阵
# cv2.getPerspectiveTransform(src, dst)
#   src：原图的4个顶点坐标（np.float32 类型）
#   dst：目标图的4个顶点坐标
# 返回值：3×3 的透视变换矩阵 M
# --------------------------------------------------------
M = cv2.getPerspectiveTransform(points, points2)

# --------------------------------------------------------
# 6️⃣ 执行透视变换
# cv2.warpPerspective(src, M, dsize[, dst[, flags[, borderMode[, borderValue]]]])
#   src：输入图像
#   M：3×3 透视变换矩阵
#   dsize：输出图像大小 (width, height)
#   flags：插值方式
#       cv2.INTER_LINEAR（默认，双线性插值）
#       cv2.INTER_CUBIC（三次插值，质量更好）
#   borderMode：边界填充模式（如 BORDER_CONSTANT 等）
#   borderValue：当边界填充时的颜色，默认黑色
# --------------------------------------------------------
image_warpPerspective = cv2.warpPerspective(
    image_np, M, (image_shape[1], image_shape[0])
)

# --------------------------------------------------------
# 7️⃣ 显示结果图像
# cv2.imshow(windowName, image)
#   windowName：窗口标题
#   image：要显示的图像
# --------------------------------------------------------
cv2.imshow("原始图像 (带标记)", image_np)
cv2.imshow("透视变换后图像", image_warpPerspective)

# --------------------------------------------------------
# 8️⃣ 等待用户按键关闭窗口
# cv2.waitKey(0)：等待键盘输入（单位毫秒，0表示无限等待）
# --------------------------------------------------------
cv2.waitKey()
cv2.destroyAllWindows()

### 🔷 OpenCV 透视变换核心知识点总结

| 项目 | 数学表达式 / 结构 | 说明 |
|------|--------------------|------|
| **透视变换定义** | $$\begin{bmatrix}x' \\ y' \\ w\end{bmatrix} = \begin{bmatrix} m_{11} & m_{12} & m_{13} \\ m_{21} & m_{22} & m_{23} \\ m_{31} & m_{32} & 1 \end{bmatrix} \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}$$ | 透视变换是一种 **3×3 的齐次矩阵映射**，能将任意四边形区域映射为另一四边形。 |
| **归一化坐标** | $$x' = \frac{m_{11}x + m_{12}y + m_{13}}{m_{31}x + m_{32}y + 1}$$ <br> $$y' = \frac{m_{21}x + m_{22}y + m_{23}}{m_{31}x + m_{32}y + 1}$$ | 通过除以第三行的权重 \(w\)，完成从齐次坐标到笛卡尔坐标的归一化。 |
| **矩阵维度** | \(3 \times 3\) | 含 8 个有效自由度（最后一个参数可取 1）。 |
| **需要匹配的点数** | 4 对点（原图与目标图） | 4 对点可解出 8 个自由度的方程组。 |
| **与仿射变换的区别** | - 仿射矩阵：\(2 \times 3\)，6 个参数<br>- 透视矩阵：\(3 \times 3\)，8 个参数 | 仿射保持平行关系，透视允许线条汇聚，有“深度感”。 |
| **函数** | `cv2.getPerspectiveTransform(src, dst)` | 计算透视矩阵 `M`，输入为两组四点坐标。 |
| **执行函数** | `cv2.warpPerspective(src, M, dsize[, flags[, borderMode[, borderValue]]])` | 根据透视矩阵 `M` 对图像执行投影变换。 |
| **核心用途** | 文档矫正、投影校正、相机标定、透视投影 | 把倾斜拍摄的画面拉正或模拟 3D 透视。 |
| **坐标变换特征** | - 平行线可相交（透视消失点）<br>- 比例随距离变化<br>- 直线保持为直线 | 符合真实视觉的透视规律。 |
| **自由度分析** | 8 个参数对应变换：<br> 平移（2）+ 旋转（1）+ 缩放（2）+ 错切（2）+ 透视（1） | 比仿射多出一个“透视投影”自由度。 |
| **透视矩阵推导** | $$\begin{cases}x'_i = \frac{m_{11}x_i + m_{12}y_i + m_{13}}{m_{31}x_i + m_{32}y_i + 1}\\[4pt]y'_i = \frac{m_{21}x_i + m_{22}y_i + m_{23}}{m_{31}x_i + m_{32}y_i + 1}\end{cases}$$ | 通过四对点建立 8 个方程，求解 \(m_{11} \dots m_{32}\)。 |
| **逆变换** | `M_inv = np.linalg.inv(M)` <br> 然后使用 `cv2.warpPerspective(img, M_inv, size)` | 可恢复原图或执行反向校正。 |
| **常见应用举例** | - 拍歪的身份证照片拉正<br>- 投影映射校正（投影仪幕布）<br>- AR 场景贴图 | 任意四边形 → 正矩形，常用于计算机视觉几何校正。 |

---

## 💡 图像变换类型对比

| 类型 | 矩阵形式 | 点数 | 自由度 | 是否保持平行 | 典型用途 |
|------|------------|--------|-----------|----------------|-------------|
| **平移** | $$\begin{bmatrix}1 & 0 & t_x \\ 0 & 1 & t_y\end{bmatrix}$$ | - | 2 | ✅ | 图像移动 |
| **仿射变换** | $$\begin{bmatrix}a_{11} & a_{12} & t_x \\ a_{21} & a_{22} & t_y\end{bmatrix}$$ | 3 对点 | 6 | ✅ | 旋转、缩放、错切 |
| **透视变换** | $$\begin{bmatrix}m_{11} & m_{12} & m_{13} \\ m_{21} & m_{22} & m_{23} \\ m_{31} & m_{32} & 1\end{bmatrix}$$ | 4 对点 | 8 | ❌ | 视角校正、文档矫正 |

---

## 📘 小结记忆

| 特性 | 仿射变换 | 透视变换 |
|------|------------|------------|
| 保平行线 | ✅ 是 | ❌ 否 |
| 点数要求 | 3 对点 | 4 对点 |
| 矩阵大小 | 2×3 | 3×3 |
| 典型用途 | 平面旋转/缩放/错切 | 投影、拉伸、视角变换 |
| 是否产生“深度” | ❌ 无 | ✅ 有 |

---

## 🧠 一句话理解
> 透视变换让二维图像“拥有了深度”——  
> 它不仅能拉直拍歪的纸，还能把平面图片变成立体视角。
